### Use CV to scan the face, read the data points


In [ ]:
# Document Description: Recognize faces and determine fatigue by analyzing yawning and blinking frequencies. Special cases like singing with a wide-open mouth or getting sand in the eyes should be avoided; both criteria need to be met to trigger an alarm.

from scipy.spatial import distance as dist
import numpy as np
import dlib
import cv2
import mysql.connector
import time
import paramiko

# MySQL Database Configuration
mysql_config = {
    'host': '192.168.137.203',
    'database': 'fatigueTest',
    'user': 'root',
    'password': 'tansome'
}

# Clear the database table
def clear_table(cursor):
    # Query the latest ten records
    select_query = 'SELECT * FROM nap ORDER BY time DESC LIMIT 10'
    cursor.execute(select_query)
    latest_data = cursor.fetchall()
    
    # Clear the table
    cursor.execute('TRUNCATE TABLE nap')
    
    # Reinsert the latest ten records into the table
    insert_query = 'INSERT INTO nap (time, blink, yawn) VALUES (%s, %s, %s)'
    for data in latest_data:
        cursor.execute(insert_query, data)

    
# Add data to the database table
def add_data(cursor, time_value, blink_value, yawn_value):
    insert_query = 'INSERT INTO nap (time, blink, yawn) VALUES (%s, %s, %s)'
    cursor.execute(insert_query, (time_value, blink_value, yawn_value))

# Connect to the MySQL database
connection = mysql.connector.connect(**mysql_config)
cursor = connection.cursor()

# Clear the database table
clear_table(cursor)

# First, automatically enable SSH and start the camera
# SSH connection information
hostname = '192.168.137.240'  # Raspberry Pi's IP address
port = 22  # SSH port number, default is 22
username = 'wwb'  # Raspberry Pi's username
password = 'pi'  # Raspberry Pi's user password

# Create an SSH client object
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    # Connect to the Raspberry Pi
    client.connect(hostname, port, username, password)
    
    # Execute the "sudo motion" command
    stdin, stdout, stderr = client.exec_command('sudo motion')

    # Get the command execution result
    output = stdout.read().decode('utf-8')
    print(output)
    
    # Close the SSH connection
    client.close()

except paramiko.AuthenticationException:
    print("Authentication failed. Please check the username and password.")

except paramiko.SSHException as ssh_err:
    print("SSH connection error:", ssh_err)

except paramiko.Exception as e:
    print("Error:", e)

# Define the URL of the video source
url = "http://192.168.137.240:8081/" # Raspberry Pi 2, used for capturing facial fatigue values
# # url = "http://mirror.aarnet.edu.au/pub/TED-talks/911Mothers_2010W-480p.mp4" # For testing purposes

# Create a video capture object
cap = cv2.VideoCapture(url)

# Check if the video capture object was opened successfully
if not cap.isOpened():
    print("Unable to open the video source")
    exit()

# Define facial landmarks
FACIAL_LANDMARKS_68_IDXS = dict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 36)),
    ("jaw", (0, 17))
])

# Function to calculate eye aspect ratio (ear)
def eye_aspect_ratio(eye):
    # Calculate distances vertically
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # Calculate distances horizontally
    C = dist.euclidean(eye[0], eye[3])
    # Calculate ear value
    ear = (A + B) / (2.0 * C)
    return ear

# Function to calculate mouth aspect ratio (mou)
def mouth_aspect_ratio(mouth):
    # Calculate distances vertically
    D = dist.euclidean(mouth[3], mouth[18])
    E = dist.euclidean(mouth[14], mouth[9])
    # Calculate distances horizontally
    F = dist.euclidean(mouth[0], mouth[6])
    # Calculate mou value
    mou = (D + E) / (2.0 * F)
    return mou 

# Function to convert shape object to NumPy array
def shape_to_np(shape, dtype="int"):
    # Create a 68x2 NumPy array
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)
    # Traverse each landmark
    # Get the coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

# Set threshold parameters (needs tuning)
EYE_AR_THRESH = 0.3  # If ear is less than 0.3, it is considered as blinking
EYE_AR_CONSEC_FRAMES = 3  # If the ear is less than 0.3 for three consecutive frames, it is considered as a blink
MOU_AR_THRESH = 0.4  # If mou is greater than 0.5, it is considered as yawning
MOU_AR_CONSEC_FRAMES = 10  # If mou is greater than 0.5 for ten consecutive frames, it is considered as yawning

# Initialize counters
COUNTER_ear = 0
COUNTER_mou = 0
TOTAL_ear = 0
TOTAL_mou = 0
start_time = time.time()

# Load the face detection and facial landmark model from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('.\shape_predictor_68_face_landmarks.dat')

# Get the regions for both eyes
(lStart, lEnd) = FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = FACIAL_LANDMARKS_68_IDXS["right_eye"]
(mStart, mEnd) = FACIAL_LANDMARKS_68_IDXS["mouth"]

# Continuously loop to detect and output results
while True:
    
    # Check if 30 seconds time interval has been reached
    current_time = time.time()
    elapsed_time = current_time - start_time
    if elapsed_time >= 30:
        # Insert the data into the MySQL database
        add_data(cursor,current_time,TOTAL_ear,TOTAL_mou)
        # Commit the database transaction
        connection.commit()

        # Reset counters and time variable
        TOTAL_ear = 0
        TOTAL_mou = 0
        start_time = time.time()
    
    # Read a video frame
    ret, frame = cap.read()
    frame = cv2.flip(frame, 0)
    # Resize the camera image to reduce computations
    frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Check if the frame was read successfully
    if not ret:
        print("Failed to retrieve video frame")
        break
        
    (h, w) = frame.shape[:2]
    width=1200
    r = width / float(w)
    dim = (width, int(h * r))
    frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    
    # Detect faces
    rects = detector(gray, 0)

    # Loop over each detected face
    for rect in rects:
        # Get the facial landmarks
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)

        # Calculate ear and mou values
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        mouth = shape[mStart:mEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        mou = mouth_aspect_ratio(mouth)

        # Calculate the average ear value
        ear = (leftEAR + rightEAR) / 2.0

        # Draw eye regions
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 0, 255), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 0, 255), 1)

        # Draw mouth region
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouth], -1, (0, 0, 255), 1)     
        
        # Check if ear meets the threshold
        if ear < EYE_AR_THRESH:
            COUNTER_ear += 1

        else:
            # If ear is less than 0.3 for several consecutive frames, count as a blink
            if COUNTER_ear >= EYE_AR_CONSEC_FRAMES:
                TOTAL_ear += 1

            # Reset the counter
            COUNTER_ear = 0

        # Check if mou meets the threshold
        if mou > MOU_AR_THRESH:
            COUNTER_mou += 1

        else:
            # If mou is greater than 0.5 for several consecutive frames, count as yawning
            if COUNTER_mou >= MOU_AR_CONSEC_FRAMES:
                TOTAL_mou += 1

            # Reset the counter
            COUNTER_mou = 0            
            
        # Display the results
        cv2.putText(frame, "Blinks: {}".format(TOTAL_ear), (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Yawns: {}".format(TOTAL_mou), (10, 60),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "MOU: {:.2f}".format(mou), (300, 60),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)        

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(10) & 0xFF
    
    # Press 'q' key to exit the loop
    if cv2.waitKey(1) == ord('q'):
        # Close the cursor and database connection
        cursor.close()
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()